In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree').getOrCreate()
data = spark.read.csv('dbfs:/FileStore/shared_uploads/gkantirisrafael@gmail.com/College.csv',inferSchema=True,header=True)

In [0]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [0]:
data.show(3)
#We will try to predict if the School is private using the other columns as features!!

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|
|  Adelphi University|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30|     12.2|         16| 10527|       56|
|      Adrian College|    Yes|1428|  1097|   336|       22|       50|       1036|         99|  

In [0]:
# Let's start to format the data
from pyspark.ml.feature import VectorAssembler
data.columns

Out[75]: ['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [0]:
#Lets build our assembler with the features that we believe are immportant!
assembler = VectorAssembler(inputCols=['Apps','Accept','Enroll','Top10perc','Top25perc','F_Undergrad','P_Undergrad','Outstate','Room_Board','Books','Personal','PhD','Terminal','S_F_Ratio',
 'perc_alumni','Expend','Grad_Rate'],outputCol='features')

In [0]:
#Let's create the output and transform the data
output = assembler.transform(data)

In [0]:
#Now we wanna deal with the'Private' column wich is a string. We will index it to numbers because SPARK ML LIBRARY cannot deal with this directly
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='Private',outputCol='Private_Index')

#We could use the Pipeline function here if there were a lot of columns to be indexed

In [0]:
#We now fit the indexed 'Private' column into the output data frame
output_fixed = indexer.fit(output).transform(output) 

In [0]:
output_fixed.printSchema()
#We only need the last 2 columns(feautes,Privare_Index) that we created

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- Private_Index: double (nullable = false)



In [0]:
#These are the only columns we actually need!
final_data = output_fixed.select('features','Private_Index')

In [0]:
#We now will split this to train data and test data
train_data , test_data = final_data.randomSplit([0.7,0.3])

In [0]:
#We will check the different classification methods that we can use
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier
#We will see how we can use the Pipeline more
from pyspark.ml import Pipeline

In [0]:
#We will create 3 different models here
dtc = DecisionTreeClassifier(labelCol='Private_Index',featuresCol='features')
rfc = RandomForestClassifier(labelCol='Private_Index',featuresCol='features',numTrees=150)
gbt = GBTClassifier(labelCol='Private_Index',featuresCol='features')

In [0]:
#We want to train all of these models.     We fit them into our train data
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [0]:
#We will check out the model comparison 
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)

In [0]:
# Now we will get some evaluation metrics and actually see how all of these do
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_binary_eval = BinaryClassificationEvaluator(labelCol='Private_Index')
my_binary_eval_2 = BinaryClassificationEvaluator(labelCol='Private_Index',rawPredictionCol='prediction')

In [0]:
#Now it's time to evaluate each of these models
print('DTC:',my_binary_eval.evaluate(dtc_predictions)) #This is the AUC
print('RFC:',my_binary_eval.evaluate(rfc_predictions))
print('GBT:',my_binary_eval.evaluate(gbt_predictions))

DTC: 0.9431221719457015
RFC: 0.9839819004524887
GBT: 0.9605882352941182


In [0]:
#We use Binary classification Evaluator right now 
#If we wanna test things like accuracy or precision
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator(labelCol='Private_Index',metricName='accuracy')
rfc_acc = acc_eval.evaluate(rfc_predictions)
rfc_acc

Out[91]: 0.9361702127659575